### Importing Needed packages


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


## Reading the data in


In [2]:
df = pd.read_csv("../../data/raw/fuel_consumption.csv")
df.head()


,MODELYEAR,MAKE,MODEL,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
0,2014,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,2014,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,2014,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,2014,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,2014,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244


Let's select some features that we want to use for regression.


In [3]:
cdf = df[["ENGINESIZE", "CYLINDERS", "FUELCONSUMPTION_COMB", "CO2EMISSIONS"]]
cdf.head(9)


,ENGINESIZE,CYLINDERS,FUELCONSUMPTION_COMB,CO2EMISSIONS
0,2.0,4,8.5,196
1,2.4,4,9.6,221
2,1.5,4,5.9,136
3,3.5,6,11.1,255
4,3.5,6,10.6,244
5,3.5,6,10.0,230
6,3.5,6,10.1,232
7,3.7,6,11.1,255
8,3.7,6,11.6,267


Let's plot Emission values with respect to Engine size:


In [4]:
fig = px.scatter(cdf, x="ENGINESIZE", y="CO2EMISSIONS", color="CYLINDERS")
fig.update_layout(
    title="Engine size vs Emission",
    xaxis_title="Engine size",
    yaxis_title="Emission",
)
fig.show()


#### Creating train and test dataset

Train/Test Split involves splitting the dataset into training and testing sets respectively, which are mutually exclusive. After which, you train with the training set and test with the testing set.


In [5]:
msk = np.random.rand(len(df)) < 0.8
train = cdf[msk]
test = cdf[~msk]


<h2 id="polynomial_regression">Polynomial regression</h2>


Sometimes, the trend of data is not really linear, and looks curvy. In this case we can use Polynomial regression methods. In fact, many different regressions exist that can be used to fit whatever the dataset looks like, such as quadratic, cubic, and so on, and it can go on and on to infinite degrees.

In essence, we can call all of these, polynomial regression, where the relationship between the independent variable x and the dependent variable y is modeled as an nth degree polynomial in x. Lets say you want to have a polynomial regression (let's make 2 degree polynomial):

$$y = b + \theta\_1  x + \theta\_2 x^2$$

Now, the question is: how we can fit our data on this equation while we have only x values, such as **Engine Size**?
Well, we can create a few additional features: 1, $x$, and $x^2$.

**PolynomialFeatures()** function in Scikit-learn library, drives a new feature sets from the original feature set. That is, a matrix will be generated consisting of all polynomial combinations of the features with degree less than or equal to the specified degree. For example, lets say the original feature set has only one feature, *ENGINESIZE*. Now, if we select the degree of the polynomial to be 2, then it generates 3 features, degree=0, degree=1 and degree=2:


In [6]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

x_train = np.asanyarray(train[["ENGINESIZE"]])
y_train = np.asanyarray(train[["CO2EMISSIONS"]])

x_test = np.asanyarray(test[["ENGINESIZE"]])
y_test = np.asanyarray(test[["CO2EMISSIONS"]])


poly = PolynomialFeatures(degree=2)
x_train_poly = poly.fit_transform(x_train)
x_train_poly


array([[ 1.  ,  2.  ,  4.  ],
       [ 1.  ,  2.4 ,  5.76],
       [ 1.  ,  1.5 ,  2.25],
       ...,
       [ 1.  ,  3.  ,  9.  ],
       [ 1.  ,  3.2 , 10.24],
       [ 1.  ,  3.2 , 10.24]])

**fit_transform** takes our x values, and output a list of our data raised from power of 0 to power of 2 (since we set the degree of our polynomial to 2).

The equation and the sample example is displayed below.

$$
\begin{bmatrix}
v\_1\\\\
v\_2\\\\
\vdots\\\\
v_n
\end{bmatrix}\longrightarrow \begin{bmatrix}
\[ 1 & v\_1 & v\_1^2]\\\\
\[ 1 & v\_2 & v\_2^2]\\\\
\vdots & \vdots & \vdots\\\\
\[ 1 & v_n & v_n^2]
\end{bmatrix}
$$

$$
\begin{bmatrix}
2.\\\\
2.4\\\\
1.5\\\\
\vdots
\end{bmatrix} \longrightarrow \begin{bmatrix}
\[ 1 & 2. & 4.]\\\\
\[ 1 & 2.4 & 5.76]\\\\
\[ 1 & 1.5 & 2.25]\\\\
\vdots & \vdots & \vdots\\\\
\end{bmatrix}
$$


It looks like feature sets for multiple linear regression analysis, right? Yes. It Does.
Indeed, Polynomial regression is a special case of linear regression, with the main idea of how do you select your features. Just consider replacing the  $x$ with $x\_1$, $x\_1^2$ with $x\_2$, and so on. Then the degree 2 equation would be turn into:

$$y = b + \theta\_1  x\_1 + \theta\_2 x\_2$$

Now, we can deal with it as 'linear regression' problem. Therefore, this polynomial regression is considered to be a special case of traditional multiple linear regression. So, you can use the same mechanism as linear regression to solve such a problems.

so we can use **LinearRegression()** function to solve it:


In [7]:
clf = linear_model.LinearRegression()
y_train_ = clf.fit(x_train_poly, y_train)
# The coefficients
print("Coefficients: ", clf.coef_)
print("Intercept: ", clf.intercept_)


Coefficients:  [[ 0.         49.10539205 -1.31985139]]
Intercept:  [109.87343696]


As mentioned before, **Coefficient** and **Intercept** , are the parameters of the fit curvy line.
Given that it is a typical multiple linear regression, with 3 parameters, and knowing that the parameters are the intercept and coefficients of hyperplane, sklearn has estimated them from our new set of feature sets. Lets plot it:


In [10]:
xx = np.arange(0.0, 10.0, 0.1)
yy = clf.intercept_[0] + clf.coef_[0][1] * xx + clf.coef_[0][2] * np.power(xx, 2)

fig = px.scatter(cdf, x="ENGINESIZE", y="CO2EMISSIONS", color="CYLINDERS")
fig.update_layout(
    title="Engine size vs Emission",
    xaxis_title="Engine size",
    yaxis_title="Emission",
)
fig.add_trace(
    go.Scatter(
        x=xx,
        y=yy,
        mode="lines",
        name="Polynomial Regression",
        line=dict(color="red", width=2),
    )
)
fig.show()


<h2 id="evaluation">Evaluation</h2>


In [11]:
from sklearn.metrics import r2_score

x_test_poly = poly.fit_transform(x_test)
y_test_ = clf.predict(x_test_poly)

print("Mean absolute error: %.2f" % np.mean(np.absolute(y_test_ - y_test)))
print("Residual sum of squares (MSE): %.2f" % np.mean((y_test_ - y_test) ** 2))
print("R2-score: %.2f" % r2_score(y_test, y_test_))


Mean absolute error: 21.57
Residual sum of squares (MSE): 818.68
R2-score: 0.76


use a polynomial regression with the dataset but this time with degree three (cubic).


In [13]:
poly3 = PolynomialFeatures(degree=3)
train_x_poly3 = poly3.fit_transform(x_train)
clf3 = linear_model.LinearRegression()
train_y3_ = clf3.fit(train_x_poly3, y_train)

# The coefficients
print("Coefficients: ", clf3.coef_)
print("Intercept: ", clf3.intercept_)

xx = np.arange(0.0, 10.0, 0.1)
yy = (
    clf3.intercept_[0]
    + clf3.coef_[0][1] * xx
    + clf3.coef_[0][2] * np.power(xx, 2)
    + clf3.coef_[0][3] * np.power(xx, 3)
)

fig = px.scatter(cdf, x="ENGINESIZE", y="CO2EMISSIONS", color="CYLINDERS")
fig.update_layout(
    title="Engine size vs Emission",
    xaxis_title="Engine size",
    yaxis_title="Emission",
)
fig.add_trace(
    go.Scatter(
        x=xx,
        y=yy,
        mode="lines",
        name="Polynomial Regression",
        line=dict(color="red", width=2),
    )
)
fig.show()

x_test_poly3 = poly3.fit_transform(x_test)
y_test_ = clf3.predict(x_test_poly3)
print("Mean absolute error: %.2f" % np.mean(np.absolute(y_test_ - y_test)))
print("Residual sum of squares (MSE): %.2f" % np.mean((y_test_ - y_test) ** 2))
print("R2-score: %.2f" % r2_score(y_test, y_test_))


Coefficients:  [[ 0.         23.43149381  5.67187861 -0.57401436]]
Intercept:  [137.48109761]


Mean absolute error: 21.75
Residual sum of squares (MSE): 825.08
R2-score: 0.76
